In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # StandardScaler 추가
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix

# 데이터 불러오기
prefix = ""  # EX: "data/"
file_list = ["stain201.csv", "stain304.csv", "stain430.csv"]
file_list = [prefix + file for file in file_list]
# 데이터 병합
data = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)
print("Data shape:", data.shape)

print(data)

In [ ]:
# X와 y 설정
y = data['pan']
X = data.drop(columns=['pan'])

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터를 이미지로 변환하기 위해 reshape
X_reshaped = X_scaled.reshape(-1, 4, 100, 1)

# 분할된 데이터셋 확인
print("X_reshaped shape:", X_reshaped.shape)

# 훈련 및 검증 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# 모델 정의
model = Sequential([
    Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(4, 100, 1)),
    MaxPooling2D(pool_size=(2, 2),padding='same'),
    Conv2D(64, kernel_size=(2, 2), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

# 모델 평가
accuracy = model.evaluate(X_test, y_test)[1]
print("Accuracy:", accuracy)

#Confusion matrix를 활용해 검증데이터를 확인하는 용도 추가하기!
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

model.save('my_model1.keras')
